# Sales Data Analysis - Key KPIs Dashboard

This notebook analyzes sales data to identify key performance indicators (KPIs) and business insights.

## Dataset Overview
- **Source**: Sales transaction data
- **Time Period**: January - March 2019
- **Locations**: Yangon, Naypyitaw, Mandalay (Branches A, B, C)
- **Product Categories**: Health & Beauty, Electronics, Home & Lifestyle, Food & Beverages, Fashion

## Key Metrics We'll Analyze
1. Total Revenue and Sales Volume
2. Branch and City Performance
3. Product Line Analysis
4. Customer Segmentation
5. Payment Method Preferences
6. Temporal Sales Patterns
7. Customer Satisfaction Metrics

In [0]:
%sql
-- Data Quality Check: Overview of our dataset
SELECT 
  COUNT(*) as total_records,
  COUNT(DISTINCT Invoice_ID) as unique_invoices,
  COUNT(DISTINCT Branch) as branches,
  COUNT(DISTINCT City) as cities,
  COUNT(DISTINCT Product_Line) as product_lines,
  MIN(Date) as earliest_date,
  MAX(Date) as latest_date
FROM `supermarket_sales`.`sales_silver`.`cleansed_sales_data`;

total_records,unique_invoices,branches,cities,product_lines,earliest_date,latest_date
1000,1000,3,3,6,2019-01-01,2019-03-30


## KPI 1: Overall Business Performance

Let's start with the most critical business metrics.

In [0]:
%sql
-- Overall Business KPIs
SELECT 
  ROUND(SUM(Total), 2) as total_revenue,
  ROUND(SUM(Gross_Income), 2) as total_gross_income,
  ROUND(SUM(COGS), 2) as total_cogs,
  COUNT(*) as total_transactions,
  SUM(Quantity) as total_items_sold,
  ROUND(AVG(Total), 2) as avg_transaction_value,
  ROUND(AVG(Rating), 2) as avg_customer_rating,
  ROUND(AVG(Gross_Margin_Percentage), 2) as avg_gross_margin_pct
FROM `supermarket_sales`.`sales_silver`.`cleansed_sales_data`;

total_revenue,total_gross_income,total_cogs,total_transactions,total_items_sold,avg_transaction_value,avg_customer_rating,avg_gross_margin_pct
322966.75,15379.37,307587.38,1000,5510,322.97,6.97,4.76


## KPI 2: Branch and Location Performance

Understanding which locations drive the most revenue and profit.

In [0]:
%sql
-- Performance by Branch and City
SELECT 
  Branch,
  City,
  COUNT(*) as transactions,
  ROUND(SUM(Total), 2) as total_revenue,
  ROUND(SUM(Gross_Income), 2) as total_profit,
  ROUND(AVG(Total), 2) as avg_transaction_value,
  ROUND(AVG(Rating), 2) as avg_rating,
  SUM(Quantity) as total_items_sold
FROM `supermarket_sales`.`sales_silver`.`cleansed_sales_data`
GROUP BY Branch, City
ORDER BY total_revenue DESC

Branch,City,transactions,total_revenue,total_profit,avg_transaction_value,avg_rating,total_items_sold
C,NAYPYITAW,328,110568.71,5265.18,337.1,7.07,1831
A,YANGON,340,106200.37,5057.16,312.35,7.03,1859
B,MANDALAY,332,106197.67,5057.03,319.87,6.82,1820


In [0]:
%sql
-- Revenue contribution by location
SELECT 
  City,
  ROUND(SUM(Total), 2) as revenue,
  ROUND(SUM(Total) * 100.0 / (SELECT SUM(Total) FROM `supermarket_sales`.`sales_silver`.`cleansed_sales_data`), 2) as revenue_percentage
FROM `supermarket_sales`.`sales_silver`.`cleansed_sales_data`
GROUP BY City
ORDER BY revenue DESC

City,revenue,revenue_percentage
NAYPYITAW,110568.71,34.24
YANGON,106200.37,32.88
MANDALAY,106197.67,32.88


## KPI 3: Product Line Analysis

Identifying top-performing product categories and their profitability.

In [0]:
%sql
-- Product Line Performance
SELECT 
  Product_Line,
  COUNT(*) as transactions,
  ROUND(SUM(Total), 2) as total_revenue,
  ROUND(SUM(Gross_Income), 2) as total_profit,
  ROUND(AVG(Total), 2) as avg_transaction_value,
  ROUND(AVG(Unit_Price), 2) as avg_unit_price,
  ROUND(AVG(Rating), 2) as avg_rating,
  SUM(Quantity) as total_quantity_sold,
  ROUND(SUM(Total) * 100.0 / (SELECT SUM(Total) FROM `supermarket_sales`.`sales_silver`.`cleansed_sales_data`), 2) as revenue_share_pct
FROM `supermarket_sales`.`sales_silver`.`cleansed_sales_data`
GROUP BY Product_Line
ORDER BY total_revenue DESC

Product_Line,transactions,total_revenue,total_profit,avg_transaction_value,avg_unit_price,avg_rating,total_quantity_sold,revenue_share_pct
Food and beverages,174,56144.84,2673.56,322.67,56.01,7.11,952,17.38
Sports and travel,166,55122.83,2624.9,332.07,56.99,6.92,920,17.07
Electronic accessories,170,54337.53,2587.5,319.63,53.55,6.92,971,16.82
Fashion accessories,178,54305.9,2586.0,305.09,57.15,7.03,902,16.81
Home and lifestyle,160,53861.91,2564.85,336.64,55.32,6.84,911,16.68
Health and beauty,152,49193.74,2342.56,323.64,54.85,7.0,854,15.23


In [0]:
%sql
-- Product Line Performance by City
SELECT 
  City,
  Product_Line,
  COUNT(*) as transactions,
  ROUND(SUM(Total), 2) as revenue,
  ROUND(AVG(Rating), 2) as avg_rating
FROM `supermarket_sales`.`sales_silver`.`cleansed_sales_data`
GROUP BY City, Product_Line
ORDER BY City, revenue DESC

City,Product_Line,transactions,revenue,avg_rating
MANDALAY,Sports and travel,62,19988.2,6.51
MANDALAY,Health and beauty,53,19980.66,7.1
MANDALAY,Home and lifestyle,50,17549.16,6.52
MANDALAY,Electronic accessories,55,17051.44,7.12
MANDALAY,Fashion accessories,62,16413.32,6.72
MANDALAY,Food and beverages,50,15214.89,6.99
NAYPYITAW,Food and beverages,66,23766.85,7.08
NAYPYITAW,Fashion accessories,65,21560.07,7.44
NAYPYITAW,Electronic accessories,55,18968.97,6.75
NAYPYITAW,Health and beauty,52,16615.33,7.0


## KPI 4: Customer Segmentation Analysis

Understanding customer behavior patterns and member vs normal customer performance.

In [0]:
%sql
-- Customer Type Analysis
SELECT 
  Customer_Type,
  COUNT(*) as transactions,
  ROUND(SUM(Total), 2) as total_revenue,
  ROUND(AVG(Total), 2) as avg_transaction_value,
  ROUND(AVG(Rating), 2) as avg_rating,
  ROUND(SUM(Total) * 100.0 / (SELECT SUM(Total) FROM `supermarket_sales`.`sales_silver`.`cleansed_sales_data`), 2) as revenue_share_pct
FROM `supermarket_sales`.`sales_silver`.`cleansed_sales_data`
GROUP BY Customer_Type
ORDER BY total_revenue DESC

Customer_Type,transactions,total_revenue,avg_transaction_value,avg_rating,revenue_share_pct
Member,501,164223.44,327.79,6.94,50.85
Normal,499,158743.31,318.12,7.01,49.15


In [0]:
%sql
-- Gender-based Analysis
SELECT 
  Gender,
  Customer_Type,
  COUNT(*) as transactions,
  ROUND(SUM(Total), 2) as total_revenue,
  ROUND(AVG(Total), 2) as avg_transaction_value,
  ROUND(AVG(Rating), 2) as avg_rating
FROM `supermarket_sales`.`sales_silver`.`cleansed_sales_data`
GROUP BY Gender, Customer_Type
ORDER BY total_revenue DESC

Gender,Customer_Type,transactions,total_revenue,avg_transaction_value,avg_rating
Female,Member,261,88146.94,337.73,6.94
Female,Normal,240,79735.98,332.23,6.99
Male,Normal,259,79007.32,305.05,7.02
Male,Member,240,76076.5,316.99,6.94


In [0]:
%sql
-- Product preference by customer segment
SELECT 
  Customer_Type,
  Product_Line,
  COUNT(*) as purchases,
  ROUND(SUM(Total), 2) as revenue,
  ROUND(AVG(Rating), 2) as avg_rating
FROM `supermarket_sales`.`sales_silver`.`cleansed_sales_data`
GROUP BY Customer_Type, Product_Line
ORDER BY Customer_Type, revenue DESC

Customer_Type,Product_Line,purchases,revenue,avg_rating
Member,Food and beverages,94,31357.62,7.0
Member,Sports and travel,87,28234.3,6.82
Member,Home and lifestyle,83,27978.03,6.87
Member,Fashion accessories,86,26323.96,6.95
Member,Health and beauty,73,25831.04,7.08
Member,Electronic accessories,78,24498.49,6.95
Normal,Electronic accessories,92,29839.04,6.9
Normal,Fashion accessories,92,27981.93,7.11
Normal,Sports and travel,79,26888.53,7.03
Normal,Home and lifestyle,77,25883.89,6.8


## KPI 5: Payment Method Preferences

Understanding customer payment behavior and its impact on business.

In [0]:
%sql
-- Payment Method Analysis
SELECT 
  Payment,
  COUNT(*) as transactions,
  ROUND(SUM(Total), 2) as total_revenue,
  ROUND(AVG(Total), 2) as avg_transaction_value,
  ROUND(AVG(Rating), 2) as avg_rating,
  ROUND(COUNT(*) * 100.0 / (SELECT COUNT(*) FROM `supermarket_sales`.`sales_silver`.`cleansed_sales_data`), 2) as usage_percentage
FROM `supermarket_sales`.`sales_silver`.`cleansed_sales_data`
GROUP BY Payment
ORDER BY transactions DESC

Payment,transactions,total_revenue,avg_transaction_value,avg_rating,usage_percentage
Ewallet,345,109993.11,318.82,6.95,34.50
Cash,344,112206.57,326.18,6.97,34.40
Credit card,311,100767.07,324.01,7.0,31.10


In [0]:
%sql
-- Payment preference by customer type and location
SELECT 
  City,
  Customer_Type,
  Payment,
  COUNT(*) as transactions,
  ROUND(SUM(Total), 2) as revenue
FROM `supermarket_sales`.`sales_silver`.`cleansed_sales_data`
GROUP BY City, Customer_Type, Payment
ORDER BY City, Customer_Type, transactions DESC

City,Customer_Type,Payment,transactions,revenue
MANDALAY,Member,Credit card,63,22607.97
MANDALAY,Member,Cash,53,16914.48
MANDALAY,Member,Ewallet,49,14182.23
MANDALAY,Normal,Ewallet,64,19331.12
MANDALAY,Normal,Cash,57,18424.98
MANDALAY,Normal,Credit card,46,14736.89
NAYPYITAW,Member,Credit card,60,19962.27
NAYPYITAW,Member,Cash,59,20063.84
NAYPYITAW,Member,Ewallet,50,16855.17
NAYPYITAW,Normal,Cash,65,23022.02


## KPI 6: Time-Based Analysis

Understanding sales patterns over time and identifying peak periods.

In [0]:
%sql
-- Monthly Sales Trends
SELECT 
  DATE_FORMAT(Date, 'yyyy-MM') as month,
  COUNT(*) as transactions,
  ROUND(SUM(Total), 2) as total_revenue,
  ROUND(SUM(Gross_Income), 2) as total_profit,
  ROUND(AVG(Total), 2) as avg_transaction_value,
  ROUND(AVG(Rating), 2) as avg_rating
FROM `supermarket_sales`.`sales_silver`.`cleansed_sales_data`
GROUP BY DATE_FORMAT(Date, 'yyyy-MM')
ORDER BY month

month,transactions,total_revenue,total_profit,avg_transaction_value,avg_rating
2019-01,352,116291.87,5537.71,330.37,7.02
2019-02,303,97219.37,4629.49,320.86,7.07
2019-03,345,109455.51,5212.17,317.26,6.84


In [0]:
%sql
-- Daily Sales Pattern (Day of Week)
SELECT 
  DAYOFWEEK(Date) as day_of_week,
  CASE 
    WHEN DAYOFWEEK(Date) = 1 THEN 'Sunday'
    WHEN DAYOFWEEK(Date) = 2 THEN 'Monday'
    WHEN DAYOFWEEK(Date) = 3 THEN 'Tuesday'
    WHEN DAYOFWEEK(Date) = 4 THEN 'Wednesday'
    WHEN DAYOFWEEK(Date) = 5 THEN 'Thursday'
    WHEN DAYOFWEEK(Date) = 6 THEN 'Friday'
    WHEN DAYOFWEEK(Date) = 7 THEN 'Saturday'
  END as day_name,
  COUNT(*) as transactions,
  ROUND(SUM(Total), 2) as total_revenue,
  ROUND(AVG(Total), 2) as avg_transaction_value
FROM `supermarket_sales`.`sales_silver`.`cleansed_sales_data`
GROUP BY DAYOFWEEK(Date)
ORDER BY day_of_week

day_of_week,day_name,transactions,total_revenue,avg_transaction_value
1,Sunday,133,44457.89,334.27
2,Monday,125,37899.08,303.19
3,Tuesday,158,51482.25,325.84
4,Wednesday,143,43731.14,305.81
5,Thursday,138,45349.25,328.62
6,Friday,139,43926.34,316.02
7,Saturday,164,56120.81,342.2


In [0]:
%sql
-- Hourly Sales Pattern
SELECT 
  HOUR(Time) as hour_of_day,
  COUNT(*) as transactions,
  ROUND(SUM(Total), 2) as total_revenue,
  ROUND(AVG(Total), 2) as avg_transaction_value
FROM `supermarket_sales`.`sales_silver`.`cleansed_sales_data`
GROUP BY HOUR(Time)
ORDER BY hour_of_day

hour_of_day,transactions,total_revenue,avg_transaction_value
10,101,31421.48,311.1
11,90,30377.33,337.53
12,89,26065.88,292.88
13,103,34723.23,337.12
14,83,30828.4,371.43
15,102,31179.51,305.68
16,77,25226.32,327.61
17,74,24445.22,330.34
18,93,26030.34,279.9
19,113,39699.51,351.32


## KPI 7: Customer Satisfaction & Quality Metrics

Analyzing customer ratings and satisfaction across different dimensions.

In [0]:
%sql
-- Rating Distribution Analysis
SELECT 
  CASE 
    WHEN Rating >= 9.0 THEN 'Excellent (9.0+)'
    WHEN Rating >= 8.0 THEN 'Very Good (8.0-8.9)'
    WHEN Rating >= 7.0 THEN 'Good (7.0-7.9)'
    WHEN Rating >= 6.0 THEN 'Fair (6.0-6.9)'
    ELSE 'Poor (<6.0)'
  END as rating_category,
  COUNT(*) as transactions,
  ROUND(COUNT(*) * 100.0 / (SELECT COUNT(*) FROM `supermarket_sales`.`sales_silver`.`cleansed_sales_data`), 2) as percentage,
  ROUND(SUM(Total), 2) as total_revenue
FROM `supermarket_sales`.`sales_silver`.`cleansed_sales_data`
GROUP BY 
  CASE 
    WHEN Rating >= 9.0 THEN 'Excellent (9.0+)'
    WHEN Rating >= 8.0 THEN 'Very Good (8.0-8.9)'
    WHEN Rating >= 7.0 THEN 'Good (7.0-7.9)'
    WHEN Rating >= 6.0 THEN 'Fair (6.0-6.9)'
    ELSE 'Poor (<6.0)'
  END


rating_category,transactions,percentage,total_revenue
Very Good (8.0-8.9),163,16.30,53519.5
Good (7.0-7.9),172,17.20,53379.15
Excellent (9.0+),166,16.60,52840.08
Poor (<6.0),315,31.50,107280.17
Fair (6.0-6.9),184,18.40,55947.84


In [0]:
%sql
-- Rating correlation with revenue
SELECT 
  City,
  Product_Line,
  ROUND(AVG(Rating), 2) as avg_rating,
  COUNT(*) as transactions,
  ROUND(SUM(Total), 2) as total_revenue,
  ROUND(AVG(Total), 2) as avg_transaction_value
FROM `supermarket_sales`.`sales_silver`.`cleansed_sales_data`
GROUP BY City, Product_Line
ORDER BY avg_rating DESC, total_revenue DESC

City,Product_Line,avg_rating,transactions,total_revenue,avg_transaction_value
NAYPYITAW,Fashion accessories,7.44,65,21560.07,331.69
YANGON,Sports and travel,7.26,59,19372.7,328.35
YANGON,Food and beverages,7.25,58,17163.1,295.92
MANDALAY,Electronic accessories,7.12,55,17051.44,310.03
MANDALAY,Health and beauty,7.1,53,19980.66,376.99
NAYPYITAW,Food and beverages,7.08,66,23766.85,360.1
NAYPYITAW,Home and lifestyle,7.06,45,13895.55,308.79
NAYPYITAW,Sports and travel,7.03,45,15761.93,350.27
NAYPYITAW,Health and beauty,7.0,52,16615.33,319.53
MANDALAY,Food and beverages,6.99,50,15214.89,304.3


## KPI 8: Profitability Analysis

Deep dive into profit margins and cost analysis.

In [0]:
%sql
-- Profitability by Product Line
SELECT 
  Product_Line,
  COUNT(*) as transactions,
  ROUND(SUM(Total), 2) as total_revenue,
  ROUND(SUM(COGS), 2) as total_costs,
  ROUND(SUM(Gross_Income), 2) as total_profit,
  ROUND(AVG(Gross_Margin_Percentage), 2) as avg_margin_pct,
  ROUND((SUM(Gross_Income) / SUM(Total)) * 100, 2) as actual_margin_pct
FROM `supermarket_sales`.`sales_silver`.`cleansed_sales_data`
GROUP BY Product_Line
ORDER BY total_profit DESC

Product_Line,transactions,total_revenue,total_costs,total_profit,avg_margin_pct,actual_margin_pct
Food and beverages,174,56144.84,53471.28,2673.56,4.76,4.76
Sports and travel,166,55122.83,52497.93,2624.9,4.76,4.76
Electronic accessories,170,54337.53,51750.03,2587.5,4.76,4.76
Fashion accessories,178,54305.9,51719.9,2586.0,4.76,4.76
Home and lifestyle,160,53861.91,51297.06,2564.85,4.76,4.76
Health and beauty,152,49193.74,46851.18,2342.56,4.76,4.76


In [0]:
%sql
-- High-value transaction analysis
SELECT 
  'High Value (>500)' as transaction_type,
  COUNT(*) as transactions,
  ROUND(SUM(Total), 2) as total_revenue,
  ROUND(AVG(Total), 2) as avg_transaction_value,
  ROUND(AVG(Rating), 2) as avg_rating
FROM `supermarket_sales`.`sales_silver`.`cleansed_sales_data`
WHERE Total > 500

UNION ALL

SELECT 
  'Medium Value (100-500)' as transaction_type,
  COUNT(*) as transactions,
  ROUND(SUM(Total), 2) as total_revenue,
  ROUND(AVG(Total), 2) as avg_transaction_value,
  ROUND(AVG(Rating), 2) as avg_rating
FROM `supermarket_sales`.`sales_silver`.`cleansed_sales_data`
WHERE Total BETWEEN 100 AND 500

UNION ALL

SELECT 
  'Low Value (<100)' as transaction_type,
  COUNT(*) as transactions,
  ROUND(SUM(Total), 2) as total_revenue,
  ROUND(AVG(Total), 2) as avg_transaction_value,
  ROUND(AVG(Rating), 2) as avg_rating
FROM `supermarket_sales`.`sales_silver`.`cleansed_sales_data`
WHERE Total < 100

ORDER BY avg_transaction_value DESC

transaction_type,transactions,total_revenue,avg_transaction_value,avg_rating
High Value (>500),227,159554.63,702.88,6.86
Medium Value (100-500),565,150299.87,266.02,7.08
Low Value (<100),208,13112.25,63.04,6.8


## KPI 9: Executive Summary & Key Insights

Final summary with actionable business insights.

In [0]:
%sql
-- Top performing combinations for strategic focus
SELECT 
  City,
  Product_Line,
  Customer_Type,
  COUNT(*) as transactions,
  ROUND(SUM(Total), 2) as revenue,
  ROUND(AVG(Total), 2) as avg_order_value,
  ROUND(AVG(Rating), 2) as avg_rating,
  ROUND(SUM(Gross_Income), 2) as profit
FROM `supermarket_sales`.`sales_silver`.`cleansed_sales_data`
GROUP BY City, Product_Line, Customer_Type
HAVING COUNT(*) >= 5  -- Filter for significant combinations
ORDER BY revenue DESC
LIMIT 15

City,Product_Line,Customer_Type,transactions,revenue,avg_order_value,avg_rating,profit
NAYPYITAW,Food and beverages,Member,36,13057.44,362.71,6.95,621.78
YANGON,Home and lifestyle,Member,34,12556.3,369.3,6.8,597.92
NAYPYITAW,Fashion accessories,Member,32,12041.85,376.31,7.64,573.42
MANDALAY,Health and beauty,Member,26,11327.82,435.69,7.18,539.42
NAYPYITAW,Electronic accessories,Normal,35,11041.02,315.46,6.95,525.76
NAYPYITAW,Food and beverages,Normal,30,10709.41,356.98,7.24,509.97
MANDALAY,Sports and travel,Member,30,10420.05,347.34,6.16,496.19
YANGON,Home and lifestyle,Normal,31,9860.9,318.09,7.07,469.57
MANDALAY,Home and lifestyle,Normal,29,9779.86,337.24,6.34,465.71
YANGON,Sports and travel,Normal,30,9695.74,323.19,7.18,461.7


In [0]:
%sql
-- Performance benchmarks and targets
WITH stats AS (
  SELECT 
    AVG(Total) as avg_transaction,
    PERCENTILE_CONT(0.5) WITHIN GROUP (ORDER BY Total) as median_transaction,
    PERCENTILE_CONT(0.75) WITHIN GROUP (ORDER BY Total) as p75_transaction,
    PERCENTILE_CONT(0.90) WITHIN GROUP (ORDER BY Total) as p90_transaction,
    AVG(Rating) as avg_rating,
    MIN(Rating) as min_rating,
    MAX(Rating) as max_rating
  FROM `supermarket_sales`.`sales_silver`.`cleansed_sales_data`
)
SELECT 
  'Transaction Value Benchmarks' as metric_type,
  ROUND(avg_transaction, 2) as average,
  ROUND(median_transaction, 2) as median,
  ROUND(p75_transaction, 2) as top_25_percent,
  ROUND(p90_transaction, 2) as top_10_percent,
  NULL as rating_avg,
  NULL as rating_range
FROM stats

UNION ALL

SELECT 
  'Customer Rating Benchmarks' as metric_type,
  NULL as average,
  NULL as median,
  NULL as top_25_percent,
  NULL as top_10_percent,
  ROUND(avg_rating, 2) as rating_avg,
  CONCAT(ROUND(min_rating, 1), ' - ', ROUND(max_rating, 1)) as rating_range
FROM stats

metric_type,average,median,top_25_percent,top_10_percent,rating_avg,rating_range
Transaction Value Benchmarks,322.97,253.85,471.35,718.91,null,null
Customer Rating Benchmarks,null,null,null,null,6.97,4.0 - 10.0


## Key Business Insights & Recommendations

Based on the analysis above, here are the critical insights:

### 🏆 Top Performers
- **Best performing city/branch combination**: Check the location performance results
- **Most profitable product line**: Review product line analysis
- **Highest-value customer segment**: Member vs Normal customer comparison

### 📊 Key Metrics Summary
1. **Revenue Concentration**: Identify which locations drive most sales
2. **Product Mix**: Understand which categories perform best
3. **Customer Loyalty**: Member vs Normal customer behavior patterns
4. **Payment Preferences**: Customer payment method adoption
5. **Time Patterns**: Peak selling hours and days
6. **Satisfaction Levels**: Rating distribution and correlation with sales

### 🎯 Strategic Recommendations
1. **Focus on high-performing locations** for expansion or increased investment
2. **Optimize product mix** based on profitability and customer ratings
3. **Develop member retention programs** if members show higher value
4. **Improve payment infrastructure** for preferred methods
5. **Staff optimization** during peak hours identified in time analysis
6. **Quality improvement** for low-rated product categories

### 📈 Growth Opportunities
- Product lines with high ratings but low volume
- Underperforming locations with improvement potential
- Customer segments with growth opportunity
- Time slots with capacity for increased sales

---
*This analysis provides a comprehensive view of business performance across multiple dimensions. Use these insights to drive data-informed decision making and strategic planning.*